In [2]:
import pandas as pd
import numpy as np

In [4]:
x_train = pd.read_csv('WaterPumpData/features_train.csv')
y_train = pd.read_csv('WaterPumpData/labels_train.csv')

In [5]:
y_train['label'] = y_train.status_group.apply(lambda x: 1 if x == 'functional' else 0)
y_train = y_train.label

In [6]:
funder = x_train.funder.str.lower().value_counts()
funder = funder[funder > 250]
x_train['funder_mod'] = [i if i in funder else 'Unknown' for i in x_train.funder]

installer = x_train.funder.str.lower().value_counts()
installer = funder[funder > 250]
installer
x_train['installer_mod'] = [i if i in installer else 'Unknown' for i in x_train.installer]

In [10]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

def pipeline(feature_set, numeric_set):
    numeric_features = numeric_set
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())])

    categorical_features =  [i for i in feature_set if i not in numeric_set]
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features),
            ('num', numeric_transformer, numeric_features)
        ])

    clf = Pipeline(steps=[('preprocessor', preprocessor)
                         ])

    clf.fit(x_train)
    
    return clf

In [11]:
feature_set = ['amount_tsh', 'population', 'funder_mod', 'district_code', 'gps_height', 'region', 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type_group', 'management_group', 'payment_type', 'quality_group', 'quantity_group', 'source_type', 'waterpoint_type_group']
numeric_features = ['amount_tsh', 'population', 'gps_height', 'construction_year']
clf = pipeline(feature_set, numeric_features)

xxx_train = pd.DataFrame(clf.transform(x_train).todense())
yyy_train = pd.DataFrame(y_train)

In [12]:
from sklearn import preprocessing
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

def create_gradientbooster_model(feature_set, cat_var):
    clf = HistGradientBoostingClassifier()
    clf.fit(xxx_train, yy_train)
    
    return clf, xxx_validate, pd.DataFrame(yy_validate)

In [13]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
def create_randomforest_model(feature_set, cat_var):
    clf = RandomForestClassifier()
    clf.fit(xxx_train, yyy_train)
    
    return clf

In [14]:
grad_boost_model = HistGradientBoostingClassifier()
grad_boost_model.fit(xxx_train, yyy_train)

/Users/prateekagarwal2/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


HistGradientBoostingClassifier()

In [15]:
random_forest_model = RandomForestClassifier()
random_forest_model.fit(xxx_train, yyy_train)

/Users/prateekagarwal2/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier()

In [16]:
from keras.models import Sequential, Model
from keras.layers import Dense, concatenate, Input
from tensorflow.keras.optimizers import Adagrad, Adam
from keras.callbacks import EarlyStopping

ip = Input(shape=xxx_train.shape[1], name='ip')
x1 = Dense(250, activation='relu')(ip)
x2 = Dense(100, activation='relu')(x1)
x3 = Dense(50, activation='relu')(x2)
x4 = Dense(10, activation='relu')(x3)
output = Dense(1, activation='sigmoid')(x4)

nn_model = Model(inputs = ip, outputs=output)
nn_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
nn_model.fit(xxx_train, yyy_train, epochs=25, batch_size = 16)

Epoch 1/25
3713/3713 [==============================] - 4s 959us/step - loss: 0.4828 - accuracy: 0.7599
Epoch 2/25
3713/3713 [==============================] - 4s 969us/step - loss: 0.4423 - accuracy: 0.7858
Epoch 3/25
3713/3713 [==============================] - 4s 975us/step - loss: 0.4261 - accuracy: 0.7952
Epoch 4/25
3713/3713 [==============================] - 4s 957us/step - loss: 0.4158 - accuracy: 0.8020
Epoch 5/25
3713/3713 [==============================] - 4s 949us/step - loss: 0.4077 - accuracy: 0.8050
Epoch 6/25
3713/3713 [==============================] - 4s 957us/step - loss: 0.3990 - accuracy: 0.8105
Epoch 7/25
3713/3713 [==============================] - 4s 952us/step - loss: 0.3942 - accuracy: 0.8136
Epoch 8/25
3713/3713 [==============================] - 4s 978us/step - loss: 0.3888 - accuracy: 0.8156
Epoch 9/25
3713/3713 [==============================] - 4s 963us/step - loss: 0.3834 - accuracy: 0.8185
Epoch 10/25
3713/3713 [==============================] - 4s 957u

In [19]:
x_test = pd.read_csv('WaterPumpData/features_test.csv')
x_test['funder_mod'] = [i if i in funder else 'Unknown' for i in x_test.funder]

x_test['installer_mod'] = [i if i in installer else 'Unknown' for i in x_test.installer]

xx_test = pd.DataFrame(clf.transform(x_test).todense())

In [20]:
grad_boost_pred = grad_boost_model.predict(xx_test)
random_forest_pred = random_forest_model.predict(xx_test)
nn_pred = nn_model.predict(xx_test)

In [25]:
y_test = pd.DataFrame(x_test['id'])

In [26]:
y_test['gb_pred'] = grad_boost_pred
y_test['rf_pred'] = random_forest_pred
y_test['nn_pred_raw'] = nn_pred

In [27]:
y_test

,id,gb_pred,rf_pred,nn_pred_raw
0,50785,0,0,5.393758e-01
1,51630,1,1,6.455615e-01
2,17168,1,1,9.317144e-01
3,45559,0,0,1.579969e-08
4,49871,1,1,9.999998e-01
...,...,...,...,...
14845,39307,0,0,1.040053e-01
14846,18990,1,1,9.993078e-01
14847,28749,1,1,8.495509e-01
14848,33492,1,1,9.377990e-01


In [28]:
y_test['nn_pred'] = round(y_test.nn_pred_raw)

In [29]:
y_test["sum"] = y_test.gb_pred + y_test.rf_pred + y_test.nn_pred

In [30]:
y_test["consensus"] = y_test['sum'].apply(lambda x: 1 if x in [2, 3] else 0)

In [31]:
y_test

,id,gb_pred,rf_pred,nn_pred_raw,nn_pred,sum,consensus
0,50785,0,0,5.393758e-01,1.0,1.0,0
1,51630,1,1,6.455615e-01,1.0,3.0,1
2,17168,1,1,9.317144e-01,1.0,3.0,1
3,45559,0,0,1.579969e-08,0.0,0.0,0
4,49871,1,1,9.999998e-01,1.0,3.0,1
...,...,...,...,...,...,...,...
14845,39307,0,0,1.040053e-01,0.0,0.0,0
14846,18990,1,1,9.993078e-01,1.0,3.0,1
14847,28749,1,1,8.495509e-01,1.0,3.0,1
14848,33492,1,1,9.377990e-01,1.0,3.0,1


In [32]:
y_test['status_group'] = y_test.consensus.apply(lambda x: 'functional' if x==1 else 'non functional')

In [33]:
results = y_test[['id', 'status_group']]
results.to_csv("submission.csv")